# Week 3: Prompt Engineering Best Practices

- Topics: Advanced prompt engineering techniques, designing effective prompts, common pitfalls, and improvement strategies.
- Hands-on: Experimenting with prompts to achieve desired responses.

## Introduction

In recent years, **prompt engineering** has emerged as a key process in the effective use of large language models. As Phoenix and Taylor (2024) describe it:

> "Prompt engineering is the process of discovering prompts that reliably yield useful or desired results."

This iterative approach focuses on refining prompts to optimize interactions with generative AI systems, achieving consistent and meaningful outputs (Phoenix and Taylor, 2024).


## Definitions of Prompt Engineering

To gain a comprehensive understanding of prompt engineering, here are several authoritative definitions from recent literature:

- **Phoenix and Taylor (2024):**
  
  "Prompt engineering is the process of discovering prompts that reliably yield useful or desired results."
  
  *(Phoenix and Taylor, 2024)*

- **Liu et al. (2023):**

  "Prompt engineering involves crafting input prompts that guide pre-trained language models to produce desired outputs, effectively leveraging their knowledge without the need for fine-tuning."

  *(Liu et al., 2023)*

- **Brown et al. (2020):**

  "Prompt engineering is the process of designing task-specific prompts to elicit desired behavior from language models, enabling them to perform a wide range of tasks without explicit training."

  *(Brown et al., 2020)*

- **Reynolds and McDonell (2021):**

  "Prompt engineering is the art of designing prompts that effectively communicate the user's intent to a language model, facilitating accurate and relevant responses."

  *(Reynolds and McDonell, 2021)*

- **White et al. (2023):**

  "Prompt engineering is a methodology for structuring inputs to large language models to achieve desired outcomes, often involving iterative refinement and understanding of model behavior."

  *(White et al., 2023)*


In [1]:
import os
import openai
from anthropic import Anthropic 
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
langchain_api_key = get_env_var("LANGCHAIN_API_KEY")
langchain_tracing_v2 = get_env_var("LANGCHAIN_TRACING_V2")
openai_api_key = get_env_var("OPENAI_API_KEY")
tavily_api_key = get_env_var("TAVILY_API_KEY")
anthropic_api_key = get_env_var("ANTHROPIC_API_KEY")

# Set up OpenAI models
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=openai_api_key)

# Direct Anthropic client
claude = Anthropic(api_key=anthropic_api_key)

# LangChain integration
claude_chat = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0, anthropic_api_key=anthropic_api_key)

## We can then use Claude in two ways:

1. Direct Anthropic client:

In [2]:
response = claude.messages.create(
    model="claude-3-5-sonnet-20240620",  # Using base model identifier
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello!"}]
)
print(response.content)

[TextBlock(text='Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.', type='text')]


2. LangChain integration (similar to how we're using OpenAI):

In [3]:
response = claude_chat.invoke("Hello!")
print(response.content)

Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.


In [ ]:
def compare_responses(prompt: str, include_claude=True, include_gpt4=True, include_gpt35=True):
    """
    Compare responses from different models for the same prompt.
    """
    results = {}

    if include_claude:
        claude_response = claude_chat.invoke(prompt)
        results["Claude-3.5-Sonnet"] = claude_response.content

    if include_gpt4:
        gpt4_response = gpt4o_chat.invoke(prompt)
        results["GPT-4o"] = gpt4_response.content

    if include_gpt35:
        gpt35_response = gpt35_chat.invoke(prompt)
        results["GPT-3.5"] = gpt35_response.content

    # Print results in a formatted way
    print(f"\nPrompt: {prompt}\n")
    print("-" * 80)
    for model, response in results.items():
        print(f"\n{model}:\n")
        print(response)
        print("-" * 80)

# Let's test with some example prompts
test_prompts = [
    "Generate a list of creative product names for a new eco-friendly cleaning brand."
]

# Run comparisons for each prompt
for prompt in test_prompts:
    compare_responses(prompt)


## References

- Phoenix, J., and Taylor, M. 2024. *Prompt Engineering for Generative AI*. Sebastopol, CA: Saxifrage, LLC, Just Understanding Data LTD, & O'Reilly Media, Inc. Available at: [O'Reilly Media](https://www.oreilly.com/library/view/prompt-engineering-for/9781098153427/)

- Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., et al. 2020. Language Models are Few-Shot Learners. In *Advances in Neural Information Processing Systems*, 33, 1877–1901. Available at: [NeurIPS Proceedings](https://proceedings.neurips.cc/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf)

- Liu, P., Yuan, W., Fu, J., Jiang, Z., Hayashi, H., and Neubig, G. 2023. Pre-train, Prompt, and Predict: A Systematic Survey of Prompting Methods in Natural Language Processing. *Trans. Assoc. Comput. Linguistics*. 11, 630–676. Available at: [ACL Anthology](https://aclanthology.org/2023.tacl-1.88.pdf)

- Reynolds, L., and McDonell, K. 2021. Prompt Programming for Large Language Models: Beyond the Few-Shot Paradigm. *arXiv preprint arXiv:2102.07350*. Available at: [arXiv](https://arxiv.org/abs/2102.07350)

- White, J., Narayanan, A., Chiu, S., Huang, Y., and Schmidt, D. 2023. A Comprehensive Survey on Prompt Engineering for Large Language Models. *arXiv preprint arXiv:2302.07842*. Available at: [arXiv](https://arxiv.org/abs/2302.07842)



